# Python进阶

## License

CC-BY-SA 4.0

大部分内容来自”a byte of python”，在此感谢作者。

# 高阶语法技巧

## 按名称传递参数

函数除了可以按照顺序传递参数，还可以按照名称传递参数。

这可以有效解决我们在“默认参数”一节提到过的问题。

即函数大量参数有默认值，导致想在某个参数上传递一个不同的值，需要将前面的参数默认值全部传递一遍的窘境。

In [1]:
from __future__ import print_function

def func(a, b=5, c=10):
    print('a is', a, 'and b is', b, 'and c is', c)

func(3, 7)
func(25, c=24)
func(c=50, a=100)

a is 3 and b is 7 and c is 10
a is 25 and b is 5 and c is 24
a is 100 and b is 5 and c is 50


## 混合多种传递方式

一般而言，比较容易理解的次序是：

1. 有按顺序传递的值，就从左向右接收数据。
2. 没有按顺序传递的值，就按名传递。
3. 找不到名字的，报错。这个名字已经按顺序传递过了的，报错。
4. 全部传递完了还没找到值的参数，用默认参数。
5. 没有默认参数的，报错。

根据上面的规则，我们可以得到一些结论：

* 定义函数时，如果带默认值参数在不带默认值的左边，报错。
* 传递值时，如果按名传递写在了按顺序传递的左边，报错。
* 设计使用函数时，如果需要想一下才知道是怎么回事的。请衡量一下这个写法是不是太繁琐了，有没有简单的办法。

## 可变参数

有时我们需要接收不确定长度个参数。例如sum函数。

或者是出于某种理由，不想处理用户的所有输入。

这时我们可以用可变参数来接收或传递参数。

可变参数的基本写法是`function_name(arguments, *argument_list, arguments_withname, **arguments_dict)`。

In [2]:
def any_arguments(*p, **kw):
    print(p)
    print(kw)

any_arguments(1, 2, 3, 4, 5, a=1, b=2, c=3, d=4, e=5)

(1, 2, 3, 4, 5)
{'a': 1, 'c': 3, 'b': 2, 'e': 5, 'd': 4}


要提防用可变参数来做list传递的行为。

In [3]:
def sum(a, b, *p):
    s = a + b
    for i in p: s += i
    return s

print(sum(1, 2))
print(sum(1, 2, 3))

3
6


In [4]:
def sum(l, inital=0):
    s = inital
    for i in l: s += i
    return s

print(sum(range(101)))

5050


## 练习

1. 查阅subprocess.Popen，写出执行一个子程序的写法。（不需要后续配合，只需要写出Popen的调用方法）
2. 写出需要改变执行目录的写法。
3. 如果你的调用在一个函数内，这个函数需要对外暴露一些Popen的细节。例如由外层来决定是否close_fds等。请写出你认为合适的写法。

## 第一类函数对象

什么叫做“第一类”？

就是“一等公民”的意思。在使用上和其他值一样，没有任何区别和歧视。

第一类函数对象的意思就是，函数对象在使用上和其他对象一样。

你是否想象过，将函数作为普通对象使用？像普通对象一样，能够定义，作为参数传递，作为结果传出。

In [5]:
op_set = {
    '+': lambda x, y: x+y,
    '-': lambda x, y: x-y,
    '*': lambda x, y: x*y,
    '/': lambda x, y: float(x)/y,
}

print(op_set['+'](1, 2))

3


## 闭包

python的作用域基本原则是，内层可以访问外层变量，外层不能访问内层。在两层的时候，只有局部和全局。

但是如果考虑在函数内可以定义函数，而且被定义出来的函数还可以作为结果值返回。

那么这个“在内层定义的函数”，除了访问自己的局部变量和全部变量外，是否可以访问定义他的函数的“局部变量”呢？

这种使得嵌套定义函数可访问上个作用域数据的定义，被称为“闭包”。

In [6]:
def addn(n):
    def add(x):
        return x+n
    return add

add10 = addn(10)
print(add10(50))

60


## 闭包的本质

正常来说，局部变量在函数结束时消亡。

然而如果在内层嵌套函数中需要访问的话，那么函数结束时局部变量也不能消亡，否则内层函数去访问什么呢？

带有闭包的函数，局部变量会被附带在刚刚生成函数上，随着这个函数对象的消亡而消亡。

因此，闭包是一种带环境数据的可执行对象。这是闭包和普通函数最大的区别。

## 闭包和类的比较

闭包是带数据的方法，类是带方法的数据，两者非常类似。可以选用简单的那个。

在需要暴露内部数据，数据附带的方法比较复杂时，建议选用类。反之，可以选用闭包。

In [7]:
class addn(object):
    def __init__(self, n):
        self.n = n
    def __call__(self, x):
        return x + self.n

add10 = addn(10)
print(add10(50))

60


## LEGB

在加入闭包后，我们可以得到完整的变量查询顺序。

局部，闭包中，全局，内置。四个名称的字母简写为LEGB。

LEGB - Local Enclosing Global Builtins

In [8]:
my_variable = 1

def func():
    my_variable = 2
    print(my_variable)
    
func()

2


In [9]:
my_variable = 1

def func():
    my_variable = 2
    def func1():
        my_variable = 3
        print(my_variable)
    return func1
    
func()()

3


## 装饰器

装饰器方法允许你重写某个函数的执行过程，在实际函数执行前后执行一些自己的辅助代码。

从而允许你将部分功能附加在其他函数上。或者运用这种手法对函数的共同行为进行剥离。

这种重写过程，可以用闭包或类来实现。

In [10]:
from __future__ import print_function

class print_arg(object):
    def __init__(self, f):
        self.f = f
    def __call__(self, *p, **kw):
        print(p, kw)
        return self.f(*p, **kw)

def add(a, b):
    return a+b
add = print_arg(add)

print(add(10, 20))

(10, 20) {}
30


In [11]:
def print_arg(f):
    def inner(*p, **kw):
        print(p, kw)
        return f(*p, **kw)
    return inner

def add(a, b):
    return a+b
add = print_arg(add)

print(add(10, 20))

(10, 20) {}
30


## 装饰器语法糖

由于装饰器在python中经常使用，因此python定义了专门的语法来方便使用。

In [12]:
@print_arg
def add(a, b):
    return a+b

print(add(10, 20))

(10, 20) {}
30


## 练习1

fib计算函数的运算很慢，因为fib在计算某个参数时，会重复执行很多次更低值的计算。

请设计一个装饰器，加速这个执行过程。并对比在加和不加这个装饰器下的效率。

In [13]:
def fib(n):
    if n <= 1: return 1
    return fib(n-1) + fib(n-2)

%time fib(35)

CPU times: user 3.66 s, sys: 0 ns, total: 3.66 s
Wall time: 3.66 s


14930352

## 答案

In [14]:
def memorized(f):
    cache = {}
    def inner(n):
        if n not in cache:
            cache[n] = f(n)
        return cache[n]
    return inner

@memorized
def fib(n):
    if n <= 1: return 1
    return fib(n-1) + fib(n-2)

%time fib(35)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 104 µs


14930352

## 练习2

请考虑上述装饰器是否能用于其他函数。

如果不能，请修改上述装饰器，使其能够用于其他函数。并说明其适用范围。

## 答案

In [15]:
def memorized(f):
    cache = {}
    def inner(*p):
        if p not in cache:
            cache[p] = f(*p)
        return cache[p]
    return inner

@memorized
def fib(n):
    if n <= 1: return 1
    return fib(n-1) + fib(n-2)

%time fib(35)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 83.9 µs


14930352

## 生成器
在python中，我们时常需要生成一个非常大的序列数据并返回。例如帖子的列表，用户的ID，等等。

正常我们都是获得数据并生成一个list返回。但是很多时候，这个序列数据是近乎于无限的。

从性能角度来说，我希望使用一个计算一个，不需要为了全部列表而耗费资源。

从使用角度来说，我希望这个列表和原来的列表使用起来没有什么太大区别。两者最好做到可以互相替换。

幸好，python为我们提供了生成器这么一种功能。

生成器等于生成数组返回。

In [16]:
def fib_seq(n):
    l = []
    a, b = 1, 1
    for i in range(n):
        a, b = a+b, a
        l.append(a)
    return l

print(fib_seq(20))

[2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711]


In [17]:
def fib_seq(n):
    a, b = 1, 1
    for i in range(n):
        a, b = a+b, a
        yield a

print(fib_seq(20))
print(list(fib_seq(20)))

<generator object fib_seq at 0x7fc8cb49ca50>
[2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711]


## 无限生成器

既然生成器机制允许我们一个个获得数据，那么我们定义一个“无限长”的list也是可以的。

只要不真的去使用所有数据，就不会产生死循环。

In [18]:
def fib_seq():
    a = b = 1
    while True:
        a, b = a+b, a
        yield b

def size_limited_seq(seq, n):
    for i in seq:
        if n <= 0: return
        yield i
        n -= 1

print(list(size_limited_seq(fib_seq(), 10)))

[1, 2, 3, 5, 8, 13, 21, 34, 55, 89]


生成器在需要时才生成。

如果此处使用数组的话，百分百会导致无法执行。

In [19]:
def number_limited_seq(seq, n):
    for i in seq:
        if i >= n: break
        yield i

print(list(number_limited_seq(fib_seq(), 100000)))

[1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025]


In [20]:
# 和前面定义的sum函数联用，sum函数支持生成器
print(sum(number_limited_seq(fib_seq(), 100000)))

196416


## map/filter/reduce

* map: 从一个列表映射到另一个列表
* filter: 从列表中选择符合条件的
* reduce: 对列表反复执行合并操作

In [21]:
# python2下无需list，但是python3下map返回生成器
print(list(map(lambda x: x*x, range(20))))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361]


In [22]:
# python2下无需list，但是python3下filter返回生成器
print(list(filter(lambda x: x % 2 == 0, range(20))))

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [23]:
# 在python2下，无需import。
# python3中，reduce已经被从全局命名空间中移除了，所以需要import。
from functools import reduce
reduce(lambda x, y: x*y, range(1, 20))

121645100408832000

## map/filter，列表推导式和for循环的比较

三者都可以完成对list操作的特定功能，例如筛选出其中某些值，将其中的值转换为另一个值。但是三者在便利程度和使用细节上略有不同。

map/filter和列表推导式比较简短，适用于相对不是很复杂的情况。真正复杂的情况需要用for来展开。

map/filter和列表推导式基本假定是“运算不彼此干扰”。即运算某个元素时，不受到其他元素的影响。for没有这一假定，某元素运算时可以受到另一个元素影响。

一般来说列表推导式比map/filter更简洁一些，因为map/filter需要定义lambda。但是在已经拥有现成函数的前提下，map/filter更加方便一些。

可以举筛法的例子。

In [24]:
print([x*x for x in range(20)])

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361]


In [25]:
l = []
for x in range(20):
    l.append(x*x)
print(l)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361]


In [26]:
print([x for x in range(20) if x % 2 == 0])

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [27]:
l = []
for x in range(20):
    if x % 2 == 0:
        l.append(x)
print(l)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


## 例子: 计算器

写一个计算器，能够计算简单的加减乘除。（不考虑优先级）

## 答案

In [28]:
op_set = {
    '+': lambda x, y: x+y,
    '-': lambda x, y: x-y,
    '*': lambda x, y: x*y,
    '/': lambda x, y: float(x)/y,
}

def parser_exp(exp):
    s = ''
    for c in exp:
        if c in op_set:
            yield s
            s = ''
            yield c
        else:
            s += c
    if s: yield s

print(list(parser_exp('1+2')))

['1', '+', '2']


In [29]:
def eval_exp(exp):
    num = None
    s = parser_exp(exp)
    for e in s:
        if e in op_set:
            r = float(next(s))
            num = op_set[e](num, r)
        else:
            num = float(e)
    return num

print(eval_exp('2*3+1'))
print(eval_exp('1+2*3'))

7.0
9.0


# 语法范式

## 引用和复制

在Python中，要分清“可变对象”和“不可变对象”。以及“对象引用”和“对象复制”。

对不可变对象而言，引用和复制并没有太大区别——反正弄来弄去都是同一个对象。

而对于可变对象，对于引用的修改会同时作用于原始对象。对复制的修改并不影响原始对象。

In [30]:
from __future__ import print_function

print('Simple Assignment')
shoplist = ['apple', 'mango', 'carrot', 'banana']
mylist = shoplist

del shoplist[0]
print('shoplist is', shoplist)
print('mylist is', mylist)
print('Copy by making a full slice')

mylist = shoplist[:]
del mylist[0]
print('shoplist is', shoplist)
print('mylist is', mylist)

Simple Assignment
shoplist is ['mango', 'carrot', 'banana']
mylist is ['mango', 'carrot', 'banana']
Copy by making a full slice
shoplist is ['mango', 'carrot', 'banana']
mylist is ['carrot', 'banana']


## 浅拷贝和深拷贝

可变对象的拷贝是一件耗时而重要的工作。

那么，如果在可变对象内有另一个可变对象呢？例如在list里嵌套了list？

只对原始的list进行复制的行为，叫做浅拷贝。对多层对象进行拷贝的行为，叫做深拷贝。

In [31]:
from __future__ import print_function
import copy

orig_list = [1, 2, 3, [4, 5, 6]]

new_list = copy.deepcopy(orig_list)
print(new_list)
new_list[3].append(7)
new_list.append(8)
print(new_list)
print(orig_list)

print('-----------')

new_list = copy.copy(orig_list)
print(new_list)
new_list[3].append(7)
new_list.append(8)
print(new_list)
print(orig_list)

[1, 2, 3, [4, 5, 6]]
[1, 2, 3, [4, 5, 6, 7], 8]
[1, 2, 3, [4, 5, 6]]
-----------
[1, 2, 3, [4, 5, 6]]
[1, 2, 3, [4, 5, 6, 7], 8]
[1, 2, 3, [4, 5, 6, 7]]


## 练习1

编写一个函数，将输入列表反序输出。

禁止用系统反向函数，例如reversed和list.reverse。

## 答案

In [32]:
l = [1,2,3,4,5]

def my_reverse(l):
    for i in range(1, len(l)+1):
        yield l[len(l)-i]

print(list(my_reverse(l)))

[5, 4, 3, 2, 1]


## 练习2

修改上面的函数，将输入列表就地反序，不产生输出。

## 答案

In [33]:
l = [1,2,3,4,5]

def my_reverse(l):
    t = l[:]
    del l[:]
    for i in range(1, len(t)+1):
        l.append(t[len(t)-i])

my_reverse(l)
print(l)

[5, 4, 3, 2, 1]


## 练习3

构造一颗自循环二叉树。

## 答案

In [34]:
l = []
l.append(l)
l.append(l)
print(l)

[[...], [...]]


## 练习4

思考一下，如果只用tuple，是否能够完成自循环树的构造。

## 多层yield范式

前面我们说过，对于list的处理常常可以归结为map/filter模式，而对于map/filter的复杂操作往往最好使用for来展开。

但是实际工作中，我们常常会发现，我们需要对list进行复杂处理，每层处理都需要用for。

而且每层处理都是可选的，总体处理过程需要根据条件进行拼装。

大家可能还记得“无限生成器”一节的size_limited_seq和number_limited_seq吧。

这种对list进行处理的模式称为链式范式——输入一个list，经过处理链条上一层层的转化，最后形成输出list。

每一个环节的输入和输出都是一个list——或其等效物生成器。因而每个环节都可以和其他环节拼装在一起。

类似的模式还在“计算器”一节出现过。

# 设计原则

## 面对对象程序设计原则

* 单一功能原则：一个对象应该仅具有一类特定功能。不要试图在上面做太多的功能。
* 开闭原则：软件体对于扩展是开放的，对于修改是封闭的。简单思考一下，如果你的代码提交后只能追加文件和修改最少的行数，如何设计？
* 里氏替换原则：对父类适用的程序，对子类也适用。这点往往会和我们日常生活的经验构成悖论。
* 接口隔离原则：多个功能独立的小接口好过一个无所不有的大接口。
* 依赖反转原则：针对接口编程而不是针对实现编程。

## 练习1

定义一些类，描述平行四边形，矩形，正方形。并按照你的理解实现继承关系。

不用实现方法和属性。

## 练习2

为刚刚的“平行四边形”，“矩形”，“正方形”类，添加以下属性和方法：

* 属性：夹角，边长
* 方法：获得夹角，设定夹角，获得边长，设定边长

## 练习3

考虑一下，刚刚你实现的类中，“设定夹角”方法，对于“长方形”和“正方形”是一个合理的方法么？

思考一下，哪个原则错了？

* 在父类能够出现的地方，一定能够使用子类替换。
* 长方形是一种平行四边形。

## 练习4

增加一个用于计算面积的函数，考虑是否可以复用呢？

## 练习5

现在实现一个方法，判断两个对象是否相等。

## Python程序设计原则

* Duck typing
* 最小惊讶原则
* 万物皆接口
* 尽早崩溃

要注意到，属性，参数名，列表结构，生成器界面，在Python中也是接口。

# 字符编码

## 字符集

* GB2312：GB 2312-80，1981年5月1日，6763个字。
* GBK：1995年12月15日，21003个字。
* GB18030：GB 18030-2005，2006年5月1日，70244个字。
* BIG5：业界标准，收入CNS 11643，13060个字。
* UNICODE：Unicode 9.0，2016年6月，128237个字。

注：资料来源，wikipedia。[国家标准代码](https://zh.wikipedia.org/wiki/%E5%9B%BD%E5%AE%B6%E6%A0%87%E5%87%86%E4%BB%A3%E7%A0%81) [Unicode](https://zh.wikipedia.org/wiki/Unicode)

GBK比CP936多了95个字。

## 编码方案

编码方案基本分为两类：变长和定长。

* GB2312/GBK使用变长编码。单字编码长度1-2字节。
* GB18030使用变长编码。单字编码长度1,2,4字节。
* UCS-2采用定长编码。单字编码长度2字节。
* UCS-2采用定长编码。单字编码长度4字节。
* UTF-8使用变长编码。单字编码长度1-3字节。（如果加上BMP之外的话为1-6）

变长类的方案在微软里称为MBCS。

## 兼容性

字符集兼容性：GB18030下行兼容GBK，GBK下行兼容GB2312。GB18030兼容Unicode3.1中日韩表意文字区。

编码兼容性：UCS-2/UCS-4互相不兼容。UTF-8独立。GB18030的编码方式下行兼容GB2312/GBK。除了UCS-2/UCS-4以外，都兼容ascii。

注意提到GBK很多地方都被写为GB2312。

## Unicode

In [35]:
b"hello world"

'hello world'

In [36]:
type(b"hello world")

str

In [37]:
u"hello world"

u'hello world'

In [38]:
type(u"hello world")

unicode

这里需要注意，Python内部的Unicode编码格式为UCS-4，而非UTF-8或者UCS-2。

## Unicode IO

In [39]:
# encoding=utf-8
import io

f = io.open("abc.txt", "wt", encoding="utf-8")
f.write(u"想象一下这里是某些中文内容，也可能是日文或者韩文")
f.close()

# 如果在windows下，这个行为反而可能出现乱码。因为windows的默认编码为CP936，而不是UTF-8。
text = io.open("abc.txt", encoding="utf-8").read()
print(text)

想象一下这里是某些中文内容，也可能是日文或者韩文


In [40]:
!ls -l abc.txt

-rw-r--r-- 1 shell shell 72 10月 13 11:49 abc.txt


In [41]:
!cat abc.txt

想象一下这里是某些中文内容，也可能是日文或者韩文

In [42]:
# encoding=utf-8
import io

f = io.open("abc.txt", "wt", encoding="gbk")
f.write(u"想象一下这里是某些中文内容，也可能是日文或者韩文")
f.close()

text = io.open("abc.txt", encoding="gbk").read()
print(text)

text = open("abc.txt").read()
print(text)

text = io.open("abc.txt", encoding="utf-8").read()
print(text)

想象一下这里是某些中文内容，也可能是日文或者韩文
����һ��������ĳЩ�������ݣ�Ҳ���������Ļ��ߺ���


UnicodeDecodeError: 'utf8' codec can't decode byte 0xcf in position 0: invalid continuation byte

In [43]:
!ls -l abc.txt

-rw-r--r-- 1 shell shell 48 10月 13 11:49 abc.txt


In [44]:
!cat abc.txt

����һ��������ĳЩ�������ݣ�Ҳ���������Ļ��ߺ���

## 练习

写一个程序，将文件从指定编码转换成指定编码。

## 答案

In [45]:
with open('abc.txt', 'rb') as fi:
    data = fi.read()
data = data.decode('gbk').encode('utf-8')
with open('abc.txt', 'wb') as fo:
    fo.write(data)

In [46]:
!cat abc.txt

想象一下这里是某些中文内容，也可能是日文或者韩文

In [47]:
!rm -f abc.txt

## 练习（可选）

为上述程序增加猜测编码功能。

注：本题目需要自行查阅chardet库的文档。这是一个猜测字符串编码的库。

## 答案

In [48]:
# encoding=utf-8
import chardet

text = u"想象一下这里是某些中文内容，也可能是日文或者韩文".encode('gbk')
print(text)

print(chardet.detect(text))
text = text.decode(chardet.detect(text)['encoding'])
print(text)
print(type(text))

����һ��������ĳЩ�������ݣ�Ҳ���������Ļ��ߺ���
{'confidence': 0.99, 'encoding': 'GB2312'}
想象一下这里是某些中文内容，也可能是日文或者韩文
<type 'unicode'>


In [49]:
# encoding=utf-8
import chardet

text = u"想象一下这里是某些中文内容，也可能是日文或者韩文".encode('utf-8')
print(text)

print(chardet.detect(text))
text = text.decode(chardet.detect(text)['encoding'])
print(text)
print(type(text))

想象一下这里是某些中文内容，也可能是日文或者韩文
{'confidence': 0.99, 'encoding': 'utf-8'}
想象一下这里是某些中文内容，也可能是日文或者韩文
<type 'unicode'>


# 正则表达式入门

如果需要将一系列内容表述成文字，我们可以很方便的使用format来达成这一目标。

然而如果是表达完成的文字解析为内容（例如int）呢？format是否有一个反操作，能让我们方便的解析内容。

当然，最傻的办法就是手写文字解析器。时间成本巨大，而且可变性很差。

我们希望一种方法，能够使用和format里的语句“差不多”的东西完整这个功能。

幸运的是，我们可以用正则表达式完成类似的功能（当然，有很大区别）。

正则表达式是用于从文本中提取内容的表达式，你可以认为他是从字符串中提取字符串的字符串。

## 从*和?说起

在搜索文件时，我们经常会用*和?表示适配一个或多个字符。

正则的思路和这个类似，但是更加复杂一些。他不但能表示匹配一个或多个字符，而且还能限定匹配哪个或者哪些字符，匹配多少。

## 匹配规则

* .       匹配除换行符以外的任意字符
* \w      匹配字母或数字或下划线或汉字
* \s      匹配任意的空白符
* \d      匹配数字
* \b      匹配单词的开始或结束
* ^       匹配字符串的开始
* $       匹配字符串的结束

## 重复次数

* \*        重复零次或更多次
* \+       重复一次或更多次
* ?       重复零次或一次
* {n}     重复n次
* {n,}    重复n次或更多次
* {n,m} 重复n到m次

## 例子

He was carefully disguised but captured quickly by police.

\w+ly

'carefully', 'quickly'

## 字符范围

可以用[...]的语法来匹配自定义内容，例如[a-z0-9]*，表示小写或数字重复任意次。

可以用^表示求反，除去这个字符不匹配。

## 捕获

在正则表达式中，可以用(...)的语法表示捕获部分内容。

例如([^/]*)，表示匹配直到/之前的所有内容，并且捕获。

被捕获的内容可以后续使用（例如用于内容替换，像sed），或者在一次匹配中或的多个内容（例如在re库中使用groups，后面会讲）。

## 非贪婪匹配

常规来说，正则会匹配“尽量长”的字符。例如我们用(h.*o)，试图匹配hello, fox中的hello。但是实际结果是hello, fo。

可以在匹配规则后面加?，表示非贪婪匹配。

此时正则会尽力匹配“尽量短”的字符。例如(h.*?o)去匹配hello, fox，此时得到hello。

## 其他材料

[正则表达式30分钟入门教程](http://deerchao.net/tutorials/regex/regex.htm)

## 练习

写出一个正则表达式，匹配手机号码。

写出一个正则表达式，匹配ipv4地址。

观看下面两个范例，解释逻辑:
    
匹配url: `(http|ftp|https):\/\/[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*)?`
                                                   
匹配email地址: `[a-z]([a-z0-9]*[-_]?[a-z0-9]+)*@([a-z0-9]*[-_]?[a-z0-9]+)+[\.][a-z]{2,3}([\.][a-z]{2})?`

# Python库示例

## 标准库:sys

In [50]:
import sys
print(sys.version_info)
print(sys.version_info.major == 3)

sys.version_info(major=2, minor=7, micro=12, releaselevel='final', serial=0)
False


## 标准库:logging

In [51]:
import os
import platform
import logging

print('normal output')

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

logging.debug("Start of the program")
logging.info("Doing something")
logging.warning("Dying now")

DEBUG:root:Start of the program
INFO:root:Doing something


normal output


## re

re是一个用来处理正则表达式的库。

* compile 将一个正则表达式“编译”。编译后的正则表达式对象比直接运行拥有更快的速度。
* search 在一系列文字中搜索。
* match 在一系列文字中匹配。
* split 利用正则将文字分裂为多个部分。
* findall 找到所有符合正则的子字符串。
* sub 对文本进行替换。

In [52]:
import re

s = 'He was carefully disguised but captured quickly by police.'
r = re.compile('\w+ly')
r.findall(s)

['carefully', 'quickly']

## search和match的区别

match一定从字符串的头开始匹配，search搜索全部文本。search即使用了^，也可能匹配到一个新行。

因此，要匹配头部，或者整个完整字符串，需要用match。而要在文字中搜索，需要用search。

细节：从复杂性上看，理所当然的，match更快。

## 练习

请写程序验证在“正则表达式”一节中所写的那些正则。

## 答案

In [53]:
import re

r = re.compile('1\d{10}')
print(r.match('13512345678'))
print(r.match('021-10101010'))

None


In [54]:
import re

r = re.compile(r'\d{0,3}\.\d{0,3}\.\d{0,3}\.\d{0,3}')
print(r.match('192.168.0.1'))
print(r.match('1200.5.4.3'))
print(r.match('abc.def.ghi.jkl'))

print(r.match('192.168.0.999'))

None
None


In [55]:
import re

# source: http://c.biancheng.net/cpp/html/1435.html
r = re.compile(r'(http|ftp|https):\/\/[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*)?')
print(r.match('http://regxlib.com/Default.aspx'))
print(r.match('regxlib.com'))

None


In [56]:
import re

# source: http://bbs.csdn.net/topics/60032440
r = re.compile(r'[a-z]([a-z0-9]*[-_]?[a-z0-9]+)*@([a-z0-9]*[-_]?[a-z0-9]+)+[\.][a-z]{2,3}([\.][a-z]{2})?')
print(r.match('shell999@gmail.com'))
print(r.match('000@shell@1'))

None


## pickle

In [57]:
import pickle
shoplistfile = 'shoplist.data'
shoplist = ['apple', 'mango', 'carrot']
f = open(shoplistfile, 'wb')
pickle.dump(shoplist, f)
f.close()
del shoplist
f = open(shoplistfile, 'rb')
storedlist = pickle.load(f)
print(storedlist)

['apple', 'mango', 'carrot']


In [58]:
!ls -l shoplist.data

-rw-r--r-- 1 shell shell 46 10月 13 11:49 shoplist.data


In [59]:
!rm shoplist.data

## datetime

* datetime.fromtimestamp
* datetime.now
* datetime.strptime
* datetime.strftime

## 练习

不要写任何代码，求出攻占攻占巴士底狱发生在周几？

提示：攻占巴士底狱发生在西历1789年7月14日。

## 答案

In [60]:
import datetime
print(datetime.date(1789, 7, 14).weekday())

1


帮助文档看[这里](https://docs.python.org/2/library/datetime.html#datetime.date.weekday)。

## math

* math.exp
* math.log
* math.pow

## 练习

算出2\*\*15有多少位数。2\*\*11515呢？2\*\*115151515呢？

## 答案

In [61]:
2**15

32768

In [62]:
len(str(2**15))

5

In [63]:
2**1515

1149326528034702581682260802282483154731794952366680598907315749676350303455258941969435226491616272765850432529872896570140083779714087641224676603265178037383658094111263859162703551253166913832114001816457662609862015656425761751871770440149072630404384442015791262449967076912697519498726922973530707535919651560412212344672313695699099755325593751836252686364599363077472034064061672117649734107141736148353976201394067293417855894280067914417745952768L

In [64]:
len(str(2**1515))

457

In [65]:
# log(2**15151515) == log2 * 15151515 == log10 * (log2/log10) * 15151515
# 2**15151515 == 10**((log2/log10) * 15151515)
import math

print((math.log(2)/math.log(10)) * 15)
print(int((math.log(2)/math.log(10)) * 15) + 1)

print((math.log(2)/math.log(10)) * 1515)
print(int((math.log(2)/math.log(10)) * 1515) + 1)

print((math.log(2)/math.log(10)) * 15151515)
print(int((math.log(2)/math.log(10)) * 15151515) + 1)

4.51544993496
5
456.060443431
457
4561060.49475
4561061


In [66]:
import math
f = lambda x: int((math.log(2)/math.log(10)) * x) + 1

print(f(15))
print(f(1515))
print(f(15151515))

5
457
4561061


## random

* randint 指定区间的随机数字
* choice 从集合中随机挑选一个
* shuffle 随机打乱集合
* random 从0-1之间随机产生一个浮点数

## 练习

做一个抽奖程序

这里要着重指出，抽奖程序是否支持重复抽奖。尤其是无放回重复抽奖。随机抽奖只需要用choice，无放回抽奖需要用shuffle。

## 答案

In [67]:
s = u'赵钱孙李周吴郑王'
s = list(s)

import random
print(random.choice(s))
print(random.choice(s))
print(random.choice(s))
print(random.choice(s))
print(random.choice(s))
print(random.choice(s))

孙
郑
郑
孙
吴
钱


In [68]:
s = u'赵钱孙李周吴郑王'
s = list(s)

import random
random.shuffle(s)
print(s.pop())
print(s.pop())
print(s.pop())
print(s.pop())
print(s.pop())
print(s.pop())

赵
钱
李
周
郑
王


## path

* basename
* dirname
* exists
* expanduser
* getsize
* join
* realpath
* splitext

## subprocess

* call
* check_output
* Popen

## 练习

完成`grep -v ^# file | wc -l`的功能，注意不能用shell调用。

用python代码完成上述功能。

注意： windows的同学无法做grep和wc，所以这道题目只能用python代码完成这部分功能。

## 答案

In [69]:
!cat /etc/rc.local

#!/bin/sh -e
#
# rc.local
#
# This script is executed at the end of each multiuser runlevel.
# Make sure that the script will "exit 0" on success or any other
# value on error.
#
# In order to enable or disable this script just change the execution
# bits.
#
# By default this script does nothing.

exit 0


In [70]:
!grep -v ^# /etc/rc.local | wc -l

2


In [71]:
# grep -v ^# file | wc -l
import subprocess

p1 = subprocess.Popen(["grep", "-v", "^#", "/etc/rc.local"], stdout=subprocess.PIPE)
p2 = subprocess.Popen(["wc", "-l"], stdin=p1.stdout, stdout=subprocess.PIPE)
output = p2.communicate()[0]
int(output.strip())

2

In [72]:
import re
import io

r = re.compile('^#')
# 在python2下无需考虑编码。
# python3下由于re需要处理string而非bytes，因此需要对输入做转码。
with io.open('/etc/rc.local', 'r', encoding='utf-8') as fi:
    print(len([line for line in fi if not r.match(line)]))

2


## pdb

使用python -m pdb file.py来启动你的某个程序。

然后试试看跟踪运行过程，看看你的解释是否正确。

如果希望在程序中快速定位到某个位置，可以考虑set_trace调用。

## unittest

执行某个代码，并且设定一些assert。如果全部正确，那么测试通过。如果出现问题，那么该子项测试失败。

unittest的好处在于，对每个小的代码段，实现一个小的自动化测试脚本，并且这些脚本可以根据框架规则整合到一起。

当代码段增多，每个代码的修改变复杂的时候，依然可以保证这些小的代码片段正确实现了当初预订的逻辑。

只要单元测试是通过的，那么每个小片段代码的逻辑就是大致上正确的。这可以有力的保证最终代码是正确的。

In [73]:
import unittest

class TestStringMethods(unittest.TestCase):

    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # check that s.split fails when the separator is not a string
        with self.assertRaises(TypeError):
            s.split(2)

suite = unittest.TestLoader().loadTestsFromTestCase(TestStringMethods)
unittest.TextTestRunner().run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

## 练习

针对“正则表达式”一节中的表达式，建立正向和反向用例，检验其正确性。

## 答案

In [74]:
import re
import unittest, unittest.loader

class TestMobile(unittest.TestCase):
    r = re.compile('1\d{10}')

    def test_match(self):
        self.assertTrue(self.r.match('13512345678'))

    def test_notmatch(self):
        self.assertFalse(self.r.match('021-10101010'))

class TestInetAddr(unittest.TestCase):
    r = re.compile(r'\d{0,3}\.\d{0,3}\.\d{0,3}\.\d{0,3}')

    def test_ipv4(self):
        self.assertTrue(self.r.match('192.168.0.1'))

    def test_notmatch(self):
        self.assertFalse(self.r.match('1200.5.4.3'))
        self.assertFalse(self.r.match('abc.def.ghi.jkl'))

    def test_wrong(self):
        # 这里会报错，这是正常的
        self.assertFalse(self.r.match('192.168.0.999'))

In [75]:
class TestUrl(unittest.TestCase):
    # source: http://c.biancheng.net/cpp/html/1435.html
    r = re.compile(r'(http|ftp|https):\/\/[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*)?')

    def test_url(self):
        self.assertTrue(self.r.match('http://regxlib.com/Default.aspx'))

    def test_domain(self):
        self.assertFalse(self.r.match('regxlib.com'))

class TestEmail(unittest.TestCase):
    # source: http://bbs.csdn.net/topics/60032440
    r = re.compile(r'[a-z]([a-z0-9]*[-_]?[a-z0-9]+)*@([a-z0-9]*[-_]?[a-z0-9]+)+[\.][a-z]{2,3}([\.][a-z]{2})?')

    def test_email(self):
        self.assertTrue(self.r.match('shell999@gmail.com'))

    def test_notmatch(self):
        self.assertFalse(self.r.match('000@shell@1'))

In [76]:
suite = unittest.TestSuite()
loader = unittest.TestLoader()
suite.addTest(loader.loadTestsFromTestCase(TestMobile))
suite.addTest(loader.loadTestsFromTestCase(TestInetAddr))
suite.addTest(loader.loadTestsFromTestCase(TestUrl))
suite.addTest(loader.loadTestsFromTestCase(TestEmail))
unittest.TextTestRunner().run(suite)

....F....
FAIL: test_wrong (__main__.TestInetAddr)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-74-ea312ac19d96>", line 25, in test_wrong
    self.assertFalse(self.r.match('192.168.0.999'))
AssertionError: <_sre.SRE_Match object at 0x7fc8c9c8b440> is not false

----------------------------------------------------------------------
Ran 9 tests in 0.006s

FAILED (failures=1)


<unittest.runner.TextTestResult run=9 errors=0 failures=1>

# 复杂例子：计算器

记得我们上面做过的练习“计算器”么？下面我们要做一个带有优先级和括号处理能力的复杂版本计算器。

请先思考一下，这类计算器如何做。然后试着实现一下想法。（假定目前只支持双目算符，但是优先级可能超过两个级别）

我实现了一下，用了20分钟。有这个时间内能够做出来的学生请务必联系我。

复杂计算器的核心问题在于算符优先级。

最简单来说，我们面对的问题核心即是a op1 b op2 c的表达式计算问题。

如果op1优先级高，我们需要计算a op1 b。如果op2高，原则上我们应当计算b op2 c。

但是实际上，由于表达式构成一个很长的链条，因此即便op2优先级比较高，也无法确定可以先算op2，有可能后续还有一个优先级更高的op3。

因此，我们需要构建一个list，并记录一个计算位置。

如果这个位置的优先级比前面的更高，那么继续向后移动。如果优先级比前面更低或相等，那么先计算前面的数据，并且将被计算部分从list中移走，插入计算结果。

在表达式结束的时候，通过持续的计算和缩减前面的数据，可以使前面的数据剩下且只剩下一项。这项即是我们需要的结果。

括号的问题更容易解决一些。

我们可以在看到(的时候，将剩余表达式作为一个新表达式传递给自身（递归）来求值，并且将)看做是一种表达式终结。

这样，()中的内容会被表达式计算函数吸收并运算成结果，替代在原始位置。经过有限次递归后，这个问题总是可以解决的。

## 答案

In [77]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
'''
@date: 2016-09-02
@author: Shell.Xu
@copyright: 2016, Shell.Xu <shell909090@gmail.com>
@license: BSD-3-clause
'''

op_set = {'(': None,
    ')': None,
    '+': lambda x, y: x+y,
    '-': lambda x, y: x-y,
    '*': lambda x, y: x*y,
    '/': lambda x, y: float(x)/y,}

op_priority = {'+': 10, '-': 10,
    '*': 20,
    '/': 20,}

def parser_exp(exp):
    s = ''
    for c in exp:
        if c in op_set:
            if s: yield s
            s = ''
            yield c
        else: s += c
    if s: yield s

In [78]:
def find_last_op(l):
    for e in reversed(l):
        if e in op_set: return e

def force_stack(stack):
    r = stack.pop(-1)
    op = stack.pop(-1)
    l = stack.pop(-1)
    result = op_set[op](l, r)
    print('{} {} {} => {}'.format(op, l, r, result))
    stack.append(result)

def eval_exp(exp):
    stack = []
    for e in exp:
        if e == ')': break
        elif e == '(':
            stack.append(eval_exp(exp))
            continue
        if e not in op_set:
            stack.append(float(e))
            continue
        while True:
            last = find_last_op(stack)
            if last is None or op_priority[last] < op_priority[e]: break
            force_stack(stack)
        stack.append(e)
    while len(stack) > 1:
        force_stack(stack)
    return stack[0]

In [79]:
def calc(exp):
    s = parser_exp(exp)
    return eval_exp(s)

print(calc('2*3+1'))
print(calc('1+2*3'))
print(calc('(1.1+2.5/10)*3/4+1'))
print(eval('(1.1+2.5/10)*3/4+1'))

* 2.0 3.0 => 6.0
+ 6.0 1.0 => 7.0
7.0
* 2.0 3.0 => 6.0
+ 1.0 6.0 => 7.0
7.0
/ 2.5 10.0 => 0.25
+ 1.1 0.25 => 1.35
* 1.35 3.0 => 4.05
/ 4.05 4.0 => 1.0125
+ 1.0125 1.0 => 2.0125
2.0125
2.0125


# pep8代码规范

* 对齐规则
* 导入规则
* 空格规则
* 注释规则
* 命名规则
* 异常规则
* 编程细节

## 对齐规则

* 使用空格而非tab，除非文件已经使用tab缩进。
* 使用4空格缩进。对于跨行而言，4空格缩进是可选的。
* 一行代码最长79个字符。对于不遵从结构的长段落（例如docstring或注释），行长度限制应当在72个字符以内。
* 跨行代码对齐同级首个元素位置，或者使用”首元素出现在上一行增加一个缩进”的对齐风格。
* 如果跨行缩进和后文缩进保持同一垂直位置，需要增加一个缩进来使其清晰。
* 跨行表达式的双目算符应出现在操作元素行首。
* 独立的顶级函数和类应该被两行空行包围。类里面的函数应被一行空行包围。
* 鼓励使用空行来分割代码中的逻辑段落，形成有节奏感的代码。

## 导入规则

* 文件应使用utf-8编码。符号应使用英文（但不排斥拼音）。
* 一行import操作一个module，但是from import多个符号是OK的。
* import应该始终在文件头部。
* import的推荐顺序是，系统库，第三方库，程序本身。
* 应使用绝对import。显式相对导入也是可以接受的。
* 泛式import（from import *）应被禁止，因为可能污染命名空间。
* 模块级dunders（例如\_\_all\_\_，\_\_version\_\_），应定义在doc string和\_\_future\_\_的后面，但是在import的前面。
* 文件的推荐次序依次是Sha-Bang，encoding声明，注释，module doc string，\_\_future\_\_ import，dunders，import，global和常量定义。

文件里唯一应当出现非ascii字符的地方，只有测试非ascii字符，和作者的名字。

## 空格规则

* 以下情况不应使用空格：
  * 括号，方括号和花括号内部的包围空格。
  * 冒号，逗号和分号前面的空格。
  * 函数和参数括号之间，列表和索引之间。
  * 批量赋值时使用超过一个空格进行对齐。
  * 行尾的空字符。
* 以下情况请务必使用空格：
  * 在赋值，比较，逻辑算符的两侧。
  * 参数默认值和按名传参中的=两侧不要使用空格。
* 以下情况建议使用算符：
  * 表达式中优先级最低的算符两侧。

## 注释规则

* 注释和代码不符比没有注释更糟糕。所以注释的有效性应被视为代码正确性的一部分。
* 对于非英语作者，请使用英文注释。除非你万分确定代码永远不会被这种语言外的使用者阅读。（否则你的注释就是无用的）
* 块注释一般和代码缩进对齐，并注释下面的代码。每行都应以#和空格开头。
* 行内注释在行的结尾处，和代码以至少两个空格分开，用于注释本行。行内注释应当以#和空格开头。
* 永远为公开的模块，函数，类和方法写docstring。
* 结束docstring的三引号应当有自己单独的一行。
* 对于单行的docstring，三引号需要保持在同一行上。

## 命名规则

* 永远不要用lIOo单独作为变量名，因为搞不清谁是谁。
* 模块建议使用短的，全小写名字。如果有助于阅读，可以用\_。包应该始终用短的，小写名字。不鼓励用\_。
* 使用C/C++模块外面包装一个py时，一般有一个前置的\_。
* 类名称建议驼峰字。
* 类型名称建议使用驼峰字。
* 异常基本上是类，所以异常应使用驼峰字。但是最后可以加Error后缀来标识。

* 函数名建议全小写。如果有助于阅读，可以用\_。如果和关键字冲突，一般会改个名字或者在后面加\_。
* 内部使用的全局变量名规则同函数名。
* 始终用self来指代对象本身，使用cls来指代类本身。
* 常量始终全大写。
* 如果要访问类属性，直接访问类属性即可，不需要额外的getter/setter。因为python允许你使用property来重载这个属性。

## 异常规则

* 从Exception继承异常而不是从BaseException继承。
* 谨慎使用异常串。
* 使用raise Exception(args)的形态而非raise Exception, args的形态。
* 不要使用泛性异常捕获，更不要用except:。
* 在捕获的时候，尽量减少无效的无效的捕获区域。以免异常遮蔽。（异常捕获的区域越小越好）
* context manager最好以独立函数或方法来进入和退出。

## 编程细节

* 代码应当被编写的尽量不妨碍其他python实现。（例如pypy，etc）所以不要依赖于一些CPython中特定的特性/优化。
* 对单例变量（例如True/False/None）的比较应始终使用is/is not，而不是比较算符。
* 使用is not而不是not … is ...。虽然逻辑意义不变，但是后者不符合英语语法。
* 使用def而不是lambda加赋值。
* return时的内容应该在函数内保持一致。
* 使用startswith, endswith而不是使用slice计算，因为前者更加符合自然语言。
* 使用isinstance而不是类型比较。

* 使用类型布尔换算来做if而非len。例如if list:，而非if len(list):。
* 不要写一个尾部有空格的多行字符串，尾部的空格对于编辑者来说是不可见的，所以部分编辑器会自动删除。这导致你的字符串出现错误。
* 不建议一行包含多个逻辑语句。
* 有的时候，对于if/for/while接一个小逻辑体，包含在一行是OK的。但是不要在多行结构里面包含逻辑体，更不要试图对这种逻辑体做跨行折叠。

# Python工程

## 代码检查工具

* pylint
* pycodestyle
* maccabe
* flake8

In [80]:
%%writefile calc.py
#!/usr/bin/python
# -*- coding: utf-8 -*-
'''
@date: 2016-09-02
@author: Shell.Xu
@copyright: 2016, Shell.Xu <shell909090@gmail.com>
@license: BSD-3-clause
'''

op_set = {'(': None,
    ')': None,
    '+': lambda x, y: x+y,
    '-': lambda x, y: x-y,
    '*': lambda x, y: x*y,
    '/': lambda x, y: float(x)/y,}

op_priority = {'+': 10, '-': 10,
    '*': 20,
    '/': 20,}

def parser_exp(exp):
    s = ''
    for c in exp:
        if c in op_set:
            if s: yield s
            s = ''
            yield c
        else: s += c
    if s: yield s

def find_last_op(l):
    for e in reversed(l):
        if e in op_set: return e

def force_stack(stack):
    r = stack.pop(-1)
    op = stack.pop(-1)
    l = stack.pop(-1)
    result = op_set[op](l, r)
    print('{} {} {} => {}'.format(op, l, r, result))
    stack.append(result)

def eval_exp(exp):
    stack = []
    for e in exp:
        if e == ')': break
        elif e == '(':
            stack.append(eval_exp(exp))
            continue
        if e not in op_set:
            stack.append(float(e))
            continue
        while True:
            last = find_last_op(stack)
            if last is None or op_priority[last] < op_priority[e]: break
            force_stack(stack)
        stack.append(e)
    while len(stack) > 1:
        force_stack(stack)
    return stack[0]

def calc(exp):
    s = parser_exp(exp)
    return eval_exp(s)

print(calc('2*3+1'))
print(calc('1+2*3'))
print(calc('(1.1+2.5/10)*3/4+1'))

Writing calc.py


In [81]:
!pylint calc.py | head -n 10

/bin/sh: pylint: 未找到命令


In [82]:
!pylint calc.py | tail -n 5

/bin/sh: pylint: 未找到命令


In [83]:
!flake8 calc.py

/bin/sh: flake8: 未找到命令


In [84]:
!python3 -m mccabe calc.py

/usr/bin/python3: No module named mccabe


In [85]:
%%writefile calc.py
#!/usr/bin/python
# -*- coding: utf-8 -*-
'''
@date: 2016-09-02
@author: Shell.Xu
@copyright: 2016, Shell.Xu <shell909090@gmail.com>
@license: BSD-3-clause
'''
from __future__ import absolute_import, division,\
    print_function, unicode_literals


op_set = {
    '(': None,
    ')': None,
    '+': lambda x, y: x+y,
    '-': lambda x, y: x-y,
    '*': lambda x, y: x*y,
    '/': lambda x, y: float(x)/y,
}


op_priority = {
    '+': 10,
    '-': 10,
    '*': 20,
    '/': 20,
}


def parser_exp(exp):
    s = ''
    for c in exp:
        if c in op_set:
            if s:
                yield s
            s = ''
            yield c
        else:
            s += c
    if s:
        yield s


def find_last_op(l):
    for e in reversed(l):
        if e in op_set:
            return e


def force_stack(stack):
    r = stack.pop(-1)
    op = stack.pop(-1)
    l = stack.pop(-1)
    result = op_set[op](l, r)
    print('{} {} {} => {}'.format(op, l, r, result))
    stack.append(result)


def eval_exp(exp):
    stack = []
    for e in exp:
        if e == ')':
            break
        elif e == '(':
            stack.append(eval_exp(exp))
            continue
        if e not in op_set:
            stack.append(float(e))
            continue
        while True:
            last = find_last_op(stack)
            if last is None or op_priority[last] < op_priority[e]:
                break
            force_stack(stack)
        stack.append(e)
    while len(stack) > 1:
        force_stack(stack)
    return stack[0]


def calc(exp):
    s = parser_exp(exp)
    result = eval_exp(s)
    assert not list(s)
    return result


print(calc('2*3+1'))
print(calc('1+2*3'))
print(calc('(1.1+2.5/10)*3/4+1'))

Overwriting calc.py


In [86]:
!pylint calc.py | head -n 10

/bin/sh: pylint: 未找到命令


In [87]:
!pylint calc.py | tail -n 5

/bin/sh: pylint: 未找到命令


In [88]:
!flake8 calc.py

/bin/sh: flake8: 未找到命令


In [89]:
!flake8 --max-complexity 10 calc.py

/bin/sh: flake8: 未找到命令


## 覆盖率分析工具

coverage

In [90]:
!python-coverage run calc.py

/bin/sh: python-coverage: 未找到命令


In [91]:
!python-coverage report -m

/bin/sh: python-coverage: 未找到命令


In [92]:
!rm -f calc.py

## 文档工具

* pydoc
* doxygen
* sphinx
* pandoc

# 编辑器设定

## 编辑器辅助功能

在开发python的时候，有一款好的编辑器往往能很大的提升工作效率。针对python来说，编辑器可能提供的特性有：

* 语法高亮
* 自动缩进
* 自动补全/类型推断
* 代码折叠/代码导航
* 符号查找和定位
* 自动错误检查
* REPL辅助

## Emacs配置

Emacs的授权是GPLv3。更多细节，可以参考[这里](https://www.emacswiki.org/emacs/PythonProgrammingInEmacs)

假定你已经选择了使用emacs作为日常编辑器，懂得如何设定配置文件和安装插件。知道如何切换到python-mode。

Emacs默认内置了python.el，自动开启了以下功能：

* 语法高亮：自动。
* 自动缩进：需要手工决定缩进深度，但是可以按tab在多个可能深度中自动巡航。
* 自动补全（半自动）：对于当前文件或工程中的符号可以自动补全，但是不会提示系统库中的符号，或者依赖的项目中的符号。
* 代码折叠：使用hs-minor-mode子模式即可完成。
* 符号定位和查找：用etags对所有文件生成一份TAGS文件，在emacs中可以借助TAGS文件做文件符号定位巡航。
* REPL：emacs内置了REPL支持。

## Vim配置

Vim是和Emacs并称的强大编辑器。Vim的授权是独特的Vim授权，但是是GPL兼容的。

* 语法高亮：下载[这个插件](http://www.vim.org/scripts/script.php?script_id=1599)，放在~/.vim/plugin/。
* 自动缩进：自动。
* 自动补全：Ctrl+n就可以补全。还可以加载pydiction进行补全。
* 代码折叠：下载[这个插件](http://vim.sourceforge.net/scripts/script.php?script_id=515)，放在~/.vim/plugin/。
  * 操作：zc折叠，zo反折叠。zN折叠所有，zn反折叠所有。
* 符号定位和查找：使用ctags -R *生成符号列表，使用Ctrl-]转跳，Ctrl-o返回。
* REPL：需设定插件，请自行研究。

更多资料，参考[这里](http://www.cnblogs.com/ifantastic/p/3185665.html)。

## pydiction的安装

1. 在[这里](http://vim.sourceforge.net/scripts/script.php?script_id=850)下载组件。
2. 然后将pydiction/after/ftplugin/python_pydiction.vim移动到~/.vim/after/ftplugin/。
3. 将pydiction/complete-dict移动到~/.vim/tools/pydiction/。
4. 在~/.vimrc里添加。


    filetype plugin on
    let g:pydiction_location = '~/.vim/tools/pydiction/complete-dict'

## Eclipse配置

Eclipse的授权为独特的EPL。Eclipse安装完成后，加装[pydev组件](http://www.pydev.org)

* 语法高亮：自动支持
* 自动缩进：自动支持
* 自动补全：支持自动补全（Ctrl+Atl+/），内置类型支持类型推断。
* 代码折叠：有功能，但无法测试通过
* 符号查找和定位：自动支持，按F3。
* REPL辅助：不支持。
* 内置语法检查。

## PyCharm配置

PyCharm是一款商业的Python IDE，专业版以上为商业授权。同时有社区版，[社区版授权](https://en.wikipedia.org/wiki/PyCharm#Licensing)为Apache License。

* 语法高亮：自动支持
* 自动缩进：自动支持
* 自动补全/类型推断：支持自动补全。
* 代码折叠/代码导航
* 符号查找和定位
* REPL辅助

## 其他编辑器

* VS code
* Atom
* Sublime
* TextMate
* Kate
* Gedit

# 2/3兼容技巧

3基本下行兼容2，2不兼容3。所以2/3兼容的基本思想就是遵循3的编写规范，同时兼顾2。

## 基本兼容性技巧

在Python文件的开始，增加以下内容（注意按照pep-8，应该放在什么位置）：

In [93]:
from __future__ import absolute_import, division, print_function, unicode_literals

这会改变几个关键性的兼容性标志。主要是print变为函数，string变为unicode。

## 编码

虽然future技巧修正了内嵌字符串，但是对于str和unicode对象而言，并没有什么变化。因此建议在需要的头部如此定义：

In [94]:
if sys.version_info.major == 3:
    unicode = str
else:
    bytes = str

Python2中定义str为裸数据，unicode为字符串，bytes没有定义。Python3中定义bytes为裸数据，str为unicode字符串，unicode没有定义。在如此定义后，可以在2/3中，统一使用bytes表示裸数据，用unicode表示字符串。

同时，在代码中，应该对所有试图表现裸数据语义的代码内嵌字符串对象，加b前缀。对所有试图表达字符串语义的代码内嵌字符串对象，加u前缀。

In [95]:
print('hello, world')
print(u'hello, world')

hello, world
hello, world


原则上，应对所有内嵌字符串对象都添加b或u前缀，以明确代码中的每一处内嵌字符串的语义。但是实际上这会大大增加工作量，而且工程实践上并没有特别大的优势。因此往往有所折衷，只在需要澄清的地方增加前缀。其他地方可以含混带过，等需要时再修改。

但是要注意到，项目越大，往往越难于根据后果追查出是哪个含混带过造成的问题。这里需要小心取舍。

## IO

Python3下文件打开时要注意是打开为二进制，还是打开为文本。即打开时的b标志位，决定了读写时应使用裸数据还是字符串。如果使用文本方式打开，还需注意编码。

Python2下没有encoding参数，因此出于兼容考虑，建议使用codecs.open。这个调用在Python2/3下的语义是一致的。

## 其余细节

* 将xrange改为range。当然，作为代价，在python2下，range的效率很有问题。
* urllib.quote和unquote改为了urllib.parse.quote和unquote。
* 某些库，在python2和python3下行为不一致，例如csv。对于这些库，尽量不要折腾兼容性。直接针对Python2/3做两种不同代码更方便。
* 使用next(it)而不是it.next()。

# 最后作业

根据当前用户目录下的~/.backup_setting.ini文件，从中读取所有需要备份的目录列表。

找出所有文件，过滤~和.bak结尾的文件，过滤特定时间之前的，过滤特定正则表达式，然后使用tar.gz打包。

再将打包结果命名为”年月日/时分秒.tar.gz”，放在指定目录下。

上述内容需要用python完成，不得使用子进程调用其他程序。

有能力的同学可以附加功能：

1. 对结果做gpg加密或签名。注意加密的密钥不应当放在机器上，因此加密和签名很可能不是同一对公私钥。
2. 可以对结果做切片。
3. 打包完成后上传到S3，然后从本地删除。